In [2]:
from beir.datasets.data_loader import GenericDataLoader

In [7]:
ClapNQcorpus, ClapNQqueries, ClapNQqrels = GenericDataLoader("/Users/sandylin/Challenge in CL/dataset").load(split="train")

100%|██████████| 183408/183408 [00:00<00:00, 365845.19it/s]


In [18]:
import json, pathlib

beir_corpus = pathlib.Path("/Users/sandylin/Challenge in CL/dataset/corpus.jsonl")
jsonl_dir   = pathlib.Path("/Users/sandylin/Challenge in CL/indexes/clapnq-jsonl")
jsonl_dir.mkdir(parents=True, exist_ok=True)
out = jsonl_dir/"docs.jsonl"

with open(beir_corpus, "r", encoding="utf-8") as fin, open(out, "w", encoding="utf-8") as fout:
    for line in fin:
        d = json.loads(line)
        contents = ((d.get("title") or "") + "\n" + (d.get("text") or "")).strip()
        fout.write(json.dumps({"id": d["_id"], "contents": contents}, ensure_ascii=False) + "\n")

print("Wrote:", out)

Wrote: /Users/sandylin/Challenge in CL/indexes/clapnq-jsonl/docs.jsonl


In [23]:
from pyserini.search.lucene import LuceneSearcher
from beir.retrieval.evaluation import EvaluateRetrieval

searcher = LuceneSearcher("/Users/sandylin/Challenge in CL/indexes/clapnq-lucene")
searcher.set_bm25(k1=0.9, b=0.4)

results = {}
for qid, qtext in ClapNQqueries.items():
    hits = searcher.search(qtext, k=10)
    results[qid] = {h.docid: float(h.score) for h in hits}

evaluator = EvaluateRetrieval(None)
ndcg, _map, recall, precision = evaluator.evaluate(ClapNQqrels, results, k_values=[1,3,5,10])
print("nDCG:", ndcg)
print("recall:", recall)

nDCG: {'NDCG@1': 0.25481, 'NDCG@3': 0.24202, 'NDCG@5': 0.25439, 'NDCG@10': 0.29381}
recall: {'Recall@1': 0.09965, 'Recall@3': 0.22929, 'Recall@5': 0.27373, 'Recall@10': 0.3678}
